In [ ]:
ls

 0H4A5595.MOV         dist.pkl                               'Sample3 Reference1.tif'
 ADIC2D.ipynb         DSLRImages/                            'Sample3 Reference2.tif'
 calibration.pkl      EDSR_x4.pt                             'Sample3 Reference.tif'
 CameraCalibration/   images/                                 speckle_pattern.jpg
 cameraMatrix.pkl     keyPointExtraction.ipynb                Test.ipynb
 correctedImage/      output_images/
 DeepDIC/            'Sample3-000 X0.00 Y0.00 N2 C0 R0.tif'


In [ ]:
cd drive

[Errno 2] No such file or directory: 'drive'
/content/drive/MyDrive/DIC


In [ ]:
cd MyDrive/

[Errno 2] No such file or directory: 'MyDrive/'
/content/drive/MyDrive/DIC


In [ ]:
cd DIC

[Errno 2] No such file or directory: 'DIC'
/content/drive/MyDrive/DIC


In [ ]:
pip install pywavelets

In [ ]:
pip install Pillow


In [ ]:
from skimage.restoration import estimate_sigma
import cv2
import pywt
import glob
import cv2
import numpy as np
from scipy.ndimage import gaussian_filter
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from scipy.ndimage import median_filter
from skimage import exposure, filters
from skimage import img_as_float64

In [ ]:


displacement_images_path = 'correctedImage/DSLR_imagesDIC/*.png'
grey_images=[]
displacement_image_paths = glob.glob(displacement_images_path)
# displacement_image_paths.append('correctedImage/corrected_refrence.png')
for i, img_path in enumerate(displacement_image_paths):
  print(img_path)
  img = cv2.imread(img_path)
  img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  # img_double = img_gray.astype(np.float64)/ 255.0
  grey_images.append(img_gray)
  cv2.imwrite(f'correctedImage/GreyImages/{img_path.split("/")[-1]}', img_gray)

correctedImage/DSLR_imagesDIC/Referencess.png


KeyboardInterrupt: 

In [ ]:
grey_images_path = 'correctedImage/GreyImages/*.png'
grey_paths = glob.glob(grey_images_path)
speckle_image=[]

sift = cv2.SIFT_create()

for i, img_path in enumerate(grey_paths):
  image = cv2.imread(img_path)
  keypoints = sift.detect(image, None)

  speckle_pattern_white = np.full_like(image, 255)

  for kp in keypoints:
        x, y = int(kp.pt[0]), int(kp.pt[1])
        cv2.circle(speckle_pattern_white, (x, y), 2, (0), -1)

    # Save the speckle pattern image
  speckle_image.append(speckle_pattern_white)

  filename_with_ext = img_path.split("/")[-1]
  filename, extension = filename_with_ext.split(".")

  # Modify the filename by inserting an underscore before the number
  modified_filename = filename[:-1] + "_" + filename[-1]

# Combine the modified filename with the extension
  new_filename = f"{modified_filename}.{extension}"
  cv2.imwrite(f'correctedImage/speckled/{new_filename}', speckle_pattern_white)



In [ ]:
import glob
orig = 'correctedImage/DSLR_imagesDIC/*.png'
orig = glob.glob(orig)
image_paths = []
Filtered_images=[]
for i, img_path in enumerate((orig)):
    print(img_path)
    # img = cv2.imread(img_path)
    # img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # # img_double = img_gray.astype(np.float64)/ 255.0
    image_paths.append(img_path)
    # Filtered_images.append(img_gray
    # cv2.imwrite(f'correctedImage/Sample1Processed/{img_path.split("/")[-1]}', img_gray)

correctedImage/DSLR_imagesDIC/Referencess.png
correctedImage/DSLR_imagesDIC/image1.png
correctedImage/DSLR_imagesDIC/image2.png
correctedImage/DSLR_imagesDIC/image3.png
correctedImage/DSLR_imagesDIC/image4.png
correctedImage/DSLR_imagesDIC/image5.png
correctedImage/DSLR_imagesDIC/image6.png
correctedImage/DSLR_imagesDIC/image7.png
correctedImage/DSLR_imagesDIC/image8.png
correctedImage/DSLR_imagesDIC/image9.png
correctedImage/DSLR_imagesDIC/image10.png
correctedImage/DSLR_imagesDIC/image11.png
correctedImage/DSLR_imagesDIC/image12.png
correctedImage/DSLR_imagesDIC/image13.png
correctedImage/DSLR_imagesDIC/image14.png
correctedImage/DSLR_imagesDIC/image15.png
correctedImage/DSLR_imagesDIC/image16.png
correctedImage/DSLR_imagesDIC/image17.png


In [ ]:
def estimate_noise(img):
  noise_level_sigma = estimate_sigma(img, average_sigmas=True)
  return noise_level_sigma

In [ ]:
def adaptive_preprocessing(img):
    """
    Perform adaptive preprocessing: Gaussian and median filtering based on estimated noise.
    """

    img = img_as_float64(image)


    noise_level = estimate_noise(img)
    print(f"Noise level: {noise_level}")


    # Convert image to PyTorch tensor
    img_tensor = torch.from_numpy(img).float().unsqueeze(0).unsqueeze(0).cuda()

    # Apply adaptive Gaussian filter
    gaussian_filtered = adaptive_gaussian_filter(img_tensor, noise_level).cpu().numpy().squeeze()
    noise_level = estimate_noise(gaussian_filtered)
    print(f"Noise level after gaussian Filter: {noise_level}")

    # # Apply adaptive median filter
    # median_filtered = adaptive_median_filter(gaussian_filtered, noise_level)

    #Apply Enhaced CLAHE
    enhanced_clahe = iterative_clahe(gaussian_filtered,iterations=5)


    noise_level_2 = estimate_noise(enhanced_clahe)
    print(f"Noise level after enhaced CLAHE: {noise_level_2}")

    enhanced_clahe_tensor = torch.from_numpy(enhanced_clahe).float().unsqueeze(0).unsqueeze(0).cuda()

    gaussian_filtered_2 = adaptive_gaussian_filter(enhanced_clahe_tensor, noise_level_2).cpu().numpy().squeeze()
    noise_level_3 = estimate_noise(gaussian_filtered_2)
    print(f"Noise level after gaussian Filter x 2: {noise_level_3}")

    return gaussian_filtered_2

In [ ]:

def gaussian_kernel(size, sigma, device):
    x = torch.arange(size, dtype=torch.float32, device=device) - (size - 1) / 2
    x = x.repeat(size, 1)
    y = x.t()
    kernel = torch.exp(-(x**2 + y**2) / (2 * sigma**2))
    kernel /= kernel.sum()
    return kernel

def adaptive_gaussian_filter(image_tensor, estimated_noise):
    """Apply adaptive Gaussian filter based on estimated noise."""
    # Ensure the input is 2D or 3D
    if image_tensor.dim() > 4:
        image_tensor = image_tensor.squeeze()

    # If the tensor is 2D, add batch and channel dimensions
    if image_tensor.dim() == 2:
        image_tensor = image_tensor.unsqueeze(0).unsqueeze(0)
    # If the tensor is 3D (assuming it's a single-channel image with batch dimension)
    elif image_tensor.dim() == 3:
        image_tensor = image_tensor.unsqueeze(1)

    sigma = max(0.3, min(2.0, estimated_noise / 10))
    kernel_size = int(2 * round(3 * sigma) + 1)

    # Create Gaussian kernel
    kernel = gaussian_kernel(kernel_size, sigma, image_tensor.device)

    #Uniform Kernel
    # kernel = torch.ones(kernel_size, kernel_size, device=image_tensor.device)
    # kernel = kernel / kernel.sum()

    padding = kernel_size // 2
    filtered = F.conv2d(image_tensor,
                        kernel.unsqueeze(0).unsqueeze(0),
                        padding=padding)

    return filtered.squeeze()

In [ ]:
def adaptive_median_filter(img, noise_level):
    initial_size = 3
    max_size = int(noise_level * 5)
    print(f"Noise level {noise_level} and max size = {max_size}")

    img_filtered = median_filter(img, size=initial_size)

    for y in range(img.shape[0]):
        for x in range(img.shape[1]):
            window_size = initial_size
            while window_size <= max_size:
                local_window = img[max(0, y-window_size//2):min(img.shape[0], y+window_size//2+1),
                                   max(0, x-window_size//2):min(img.shape[1], x+window_size//2+1)]
                local_median = np.median(local_window)
                if abs(img[y, x] - local_median) < noise_level:
                    img_filtered[y, x] = local_median
                    break
                window_size += 2

    return img_filtered

In [ ]:
# def enhanced_adaptive_clahe(image, noise_level, base_clip_limit=2.0, base_tile_size=8 ):
#     """
#     Apply further optimized Adaptive CLAHE.

#     :param image: Input image (2D numpy array)
#     :param base_clip_limit: Base clipping limit for CLAHE
#     :param base_tile_size: Base tile size for CLAHE
#     :return: Optimized CLAHE-enhanced image
#     """
#     # Convert image to 8-bit if necessary
#     if image.dtype != np.uint8:
#         image = exposure.rescale_intensity(image, out_range=(0, 255)).astype(np.uint8)

#     # # Multi-scale noise estimation (example: using Gaussian blur)
#     # noise_level = estimate_sigma(image, average_sigmas=True)

#     # Edge density estimation (using Sobel filter or other methods)
#     edges = filters.sobel(image)
#     edge_density = np.mean(edges)

#     # Dynamic adjustment of clip limit and tile size
#     adjusted_clip_limit = base_clip_limit * (1 + edge_density) / (1 + noise_level)
#     adjusted_clip_limit = np.clip(adjusted_clip_limit, 0.8, 4.0)

#     image_size = min(image.shape)
#     size_factor = image_size / 512
#     adjusted_tile_size = int(base_tile_size * size_factor * (1 / (1 + edge_density)))
#     adjusted_tile_size = max(4, min(adjusted_tile_size, 16))

#     # Apply CLAHE
#     clahe = cv2.createCLAHE(clipLimit=adjusted_clip_limit,
#                             tileGridSize=(adjusted_tile_size, adjusted_tile_size))
#     enhanced_image = clahe.apply(image)

#     print(f"Enhanced CLAHE parameters - Clip Limit: {adjusted_clip_limit:.2f}, Tile Size: {adjusted_tile_size}")

#     return enhanced_image


def iterative_clahe(image, base_clip_limit=2.0, base_tile_size=8, iterations=5):
    """
    Apply iterative CLAHE with adjustments based on the previous result.

    :param image: Input image (2D numpy array)
    :param base_clip_limit: Base clipping limit for CLAHE
    :param base_tile_size: Base tile size for CLAHE
    :param iterations: Number of iterations to perform
    :return: Iteratively enhanced image
    """

    # Convert image to 8-bit if necessary
    if image.dtype != np.uint8:
        image = exposure.rescale_intensity(image, out_range=(0, 255)).astype(np.uint8)

    current_image = image.copy()

    for i in range(iterations):
        noise_level = estimate_sigma(current_image, average_sigmas=True)
        print(f"Iteration {i+1}: Noise level = {noise_level}")
        edges = filters.sobel(current_image)
        edge_density = np.mean(edges)

        adjusted_clip_limit = base_clip_limit * (1 + edge_density) / (1 + noise_level)
        adjusted_clip_limit = np.clip(adjusted_clip_limit, 0.8, 4.0)

        image_size = min(current_image.shape)
        size_factor = image_size / 512
        adjusted_tile_size = int(base_tile_size * size_factor * (1 / (1 + edge_density)))
        adjusted_tile_size = max(4, min(adjusted_tile_size, 16))

        clahe = cv2.createCLAHE(clipLimit=adjusted_clip_limit,
                                tileGridSize=(adjusted_tile_size, adjusted_tile_size))
        current_image = clahe.apply(current_image)

        print(f"Iteration {i+1}: Clip Limit = {adjusted_clip_limit:.2f}, Tile Size = {adjusted_tile_size}")

    return current_image



In [ ]:

# Process all images again
processed_images = []
sigma_estimates = []
gaussian_sigmas = []

for path in image_paths:
    image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    print(f"type = {image.dtype}")
    filtered_image= adaptive_preprocessing(image)
    processed_images.append(filtered_image)
# Display the original and filtered images for comparison


for i, (original_path, processed_image) in enumerate(zip(image_paths, processed_images)):
    original_image = cv2.imread(original_path, cv2.IMREAD_GRAYSCALE)
    print(f"Image {i+1}: shape={processed_image.shape}, dtype={processed_image.dtype}, min={processed_image.min()}, max={processed_image.max()}, path={original_path}")
    plt.figure(figsize=(10, 10))
    plt.subplot(1, 2, 1)
    plt.imshow(original_image, cmap='gray')
    plt.title('Original Image')
    plt.axis('off')

    #img = to_torch(img)
    # Filtered_images.append(filtered_img);
    plt.subplot(1, 2, 2)
    plt.imshow(processed_image, cmap='gray')
    plt.title('Processed Image')
    plt.axis('off')
    plt.show()
    print(f"i= {i}")
# Return the sigma estimates and applied Gaussian sigmas for further analysis


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
for i, img in enumerate(processed_images):
  cv2.imwrite(f'correctedImage/Sample1Processed/SampleImage_{i}.tif', img)

In [ ]:
import cv2
import os

# Directory path where the images will be saved
save_directory = 'correctedImage/DSLR1Processed/'

# Create the directory if it doesn't exist
os.makedirs(save_directory, exist_ok=True)

# Loop through the processed images and save them
for i, img in enumerate(processed_images):
    # Ensure the image is in the correct format (uint8 or uint16)
    if img.dtype != np.uint8 and img.dtype != np.uint16:
        img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)  # Normalize and convert to uint8

    # Save the image with a unique filename
    cv2.imwrite(os.path.join(save_directory, f'SampleImage_{i}.tif'), img)

print("All images have been saved successfully.")


All images have been saved successfully.
